In [1]:
import pandas as pd

df_dance = pd.read_csv('../data/dataset_0.csv')
df_idle = pd.read_csv('../data/dataset_1.csv')
df_walk = pd.read_csv('../data/dataset_2.csv')
df_run = pd.read_csv('../data/dataset_3.csv')
df_spin = pd.read_csv('../data/dataset_4.csv')

In [2]:
df = pd.concat([df_dance, df_idle, df_walk, df_run, df_spin])
df.rename(columns={'Set': 'Label'}, inplace=True)
df.drop(columns=['OrientationX_sin', 'OrientationX_cos', 'OrientationY_sin', 'OrientationY_cos', 'OrientationZ_sin', 'OrientationZ_cos'], inplace=True) #Let´s try to use only accelerometer and gyroscope at least for now

In [3]:
df.head()

,Label,AnimIndex,AccelerationX,AccelerationY,AccelerationZ,GyroscopeX,GyroscopeY,GyroscopeZ
0,Dance,0,-0.029801,0.106064,0.015994,0.164124,-0.018282,-0.041787
1,Dance,0,39.974822,-103.035792,-41.127805,9.177075,0.814839,0.038732
2,Dance,0,39.993631,-103.234790,-40.895767,8.912522,0.704715,-0.185318
3,Dance,0,-34.561648,103.504571,40.498262,-0.039133,-1.434110,-0.152650
4,Dance,0,0.560703,-4.028138,-0.039327,-0.488945,-0.616965,0.132907


In [4]:
from sklearn.preprocessing import StandardScaler

feature_cols = ['AccelerationX', 'AccelerationY', 'AccelerationZ', 'GyroscopeX', 'GyroscopeY', 'GyroscopeZ']
scaler = StandardScaler()
df[feature_cols] = scaler.fit_transform(df[feature_cols])

C:\Users\felip\AppData\Roaming\Python\Python310\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
import numpy as np

def create_sliding_windows(data, label_col, window_size, step_size):
    sequences = []
    labels = []
    for start in range(0, len(data) - window_size + 1, step_size):
        end = start + window_size
        window = data.iloc[start:end]
        sequences.append(window[feature_cols].values)
        label = window[label_col].mode()[0]
        labels.append(label)
    return np.array(sequences), np.array(labels)

window_size = 100
step_size = 50
X, y = create_sliding_windows(df, label_col='Label', window_size=window_size, step_size=step_size)

In [6]:
from sklearn.model_selection import train_test_split

train_indices, temp_indices = train_test_split(df['AnimIndex'].unique(), test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42)

train_data = df[df['AnimIndex'].isin(train_indices)]
val_data = df[df['AnimIndex'].isin(val_indices)]
test_data = df[df['AnimIndex'].isin(test_indices)]

X_train, y_train = create_sliding_windows(train_data, label_col='Label', window_size=window_size, step_size=step_size)
X_val, y_val = create_sliding_windows(val_data, label_col='Label', window_size=window_size, step_size=step_size)
X_test, y_test = create_sliding_windows(test_data, label_col='Label', window_size=window_size, step_size=step_size)

In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, input_shape=(window_size, len(feature_cols)), return_sequences=False),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                18176     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 5)                 325       
                                                                 
Total params: 22,661
Trainable params: 22,661
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10


ValueError: in user code:

    File "C:\Users\felip\AppData\Roaming\Python\Python310\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\felip\AppData\Roaming\Python\Python310\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\felip\AppData\Roaming\Python\Python310\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\felip\AppData\Roaming\Python\Python310\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\felip\AppData\Roaming\Python\Python310\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "C:\Users\felip\AppData\Roaming\Python\Python310\site-packages\keras\en

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')